In [354]:
# Import necessary libraries
import pandas as pd
import os
import pycountry

# 1. Extracting Data
## Write code to extract the 31 country files from the emissions dataset that you are using for the project.

In [355]:
# Dataframe for sector
sector_data = {'sectorName': ['agriculture', 'buildings', 'fluorinated_gases', 'fossil_fuel_operations', 'manufacturing', 'mineral_extraction', 'power', 'waste']}
dimSector = pd.DataFrame(sector_data)
dimSector.insert(0, 'sectorID', range(1, len(dimSector) + 1))

In [356]:
dimSector

,sectorID,sectorName
0,1,agriculture
1,2,buildings
2,3,fluorinated_gases
3,4,fossil_fuel_operations
4,5,manufacturing
5,6,mineral_extraction
6,7,power
7,8,waste


In [357]:
# Dataframe for subsector
subsector_data = {
    'subsectorName': [
        # Agriculture
        'cropland-fires', 'enteric-fermentation', 'manure-management',
        'other-agricultural-soil-emissions', 'rice-cultivation', 'synthetic-fertilizer-application',
        
        # Buildings
        'other-onsite-fuel-usage', 'residential-and-commercial-onsite-fuel-usage',
        
        # Fluorinated Gases
        'fluorinated-gases',
        
        # Fossil Fuel Operations
        'coal-mining', 'oil-and-gas-production-and-transport', 'oil-and-gas-refining',
        'other-fossil-fuel-operations', 'solid-fuel-transformation',
        
        # Manufacturing
        'aluminum', 'cement', 'chemicals', 'other-manufacturing', 'pulp-and-paper', 'steel',
        
        # Mineral Extraction
        'bauxite-mining', 'copper-mining', 'iron-mining', 'rock-quarrying', 'sand-quarrying',
        
        # Power
        'electricity-generation', 'other-energy-use',
        
        # Waste
        'biological-treatment-of-solid-waste-&-biogenic', 'incineration-and-open-burning-of-waste',
        'solid-waste-disposal', 'wastewater-treatment-and-discharge'
    ],
    'sectorName': [
        # Agriculture
        'agriculture', 'agriculture', 'agriculture',
        'agriculture', 'agriculture', 'agriculture',
        
        # Buildings
        'buildings', 'buildings',
        
        # Fluorinated Gases
        'fluorinated_gases',
        
        # Fossil Fuel Operations
        'fossil_fuel_operations', 'fossil_fuel_operations', 'fossil_fuel_operations',
        'fossil_fuel_operations', 'fossil_fuel_operations',
        
        # Manufacturing
        'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing',
        
        # Mineral Extraction
        'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction',
        
        # Power
        'power', 'power',
        
        # Waste
        'waste', 'waste', 'waste', 'waste'
    ]
}

dimSubSector = pd.DataFrame(subsector_data)
dimSubSector = dimSubSector.merge(dimSector, on='sectorName')
dimSubSector.insert(0, 'subsectorID', range(1, len(dimSubSector) + 1))
dimSubSector.drop(columns=['sectorName'], inplace=True)
dimSubSector = dimSubSector[['subsectorID', 'sectorID', 'subsectorName']]

In [358]:
dimSubSector.head(10)

,subsectorID,sectorID,subsectorName
0,1,1,cropland-fires
1,2,1,enteric-fermentation
2,3,1,manure-management
3,4,1,other-agricultural-soil-emissions
4,5,1,rice-cultivation
5,6,1,synthetic-fertilizer-application
6,7,2,other-onsite-fuel-usage
7,8,2,residential-and-commercial-onsite-fuel-usage
8,9,3,fluorinated-gases
9,10,4,coal-mining


In [359]:
# Combined csv will all the data
main_directory = 'original_data/'
dataframes = []

for root, dirs, files in os.walk(main_directory):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

In [360]:
combined_df.head()

,iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date
0,MKD,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
1,MKD,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,ch4,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
2,MLI,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
3,MLI,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,ch4,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
4,MLT,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN


In [361]:
# Dataframe for country
dimCountry = combined_df['iso3_country'].unique()
dimCountry = pd.DataFrame(dimCountry, columns=['countryCode'])
dimCountry.insert(0, 'countryID', range(1, len(dimCountry) + 1))

def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        return 'Unknown'

dimCountry['countryName'] = dimCountry['countryCode'].apply(get_country_name)

In [362]:
dimCountry

,countryID,countryCode,countryName
0,1,MKD,North Macedonia
1,2,MLI,Mali
2,3,MLT,Malta
3,4,MMR,Myanmar
4,5,ABW,Aruba
...,...,...,...
246,247,VAT,Holy See (Vatican City State)
247,248,CCK,Cocos (Keeling) Islands
248,249,CXR,Christmas Island
249,250,SGS,South Georgia and the South Sandwich Islands


In [363]:
# Dataframe for gas
dimGas = combined_df['gas'].unique()
dimGas = pd.DataFrame(dimGas, columns=['gasName'])
dimGas.insert(0, 'gasID', range(1, len(dimGas) + 1))

In [364]:
dimGas

,gasID,gasName
0,1,co2
1,2,ch4
2,3,n2o
3,4,co2e_20yr
4,5,co2e_100yr


In [365]:
# Dataframe for year 
unique_years = combined_df['start_time'].astype(str).str[:4].unique()
unique_years = sorted(unique_years)
dimYear = pd.DataFrame(unique_years, columns=['yearName'])
dimYear.insert(0, 'yearID', range(1, len(dimYear) + 1))

In [366]:
dimYear

,yearID,yearName
0,1,2015
1,2,2016
2,3,2017
3,4,2018
4,5,2019
5,6,2020
6,7,2021
7,8,2022


# 2.1. Cleaning the data

In [367]:
# fix countries that were not mapped correctly
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryID,countryCode,countryName
221,222,SCG,Unknown
245,246,XKX,Unknown


In [368]:
dimCountry.loc[dimCountry['countryCode'] == 'SCG', 'countryName'] = 'Serbia and Montenegro'
dimCountry.loc[dimCountry['countryCode'] == 'XKX', 'countryName'] = 'Republic of Kosovo'
dimCountry.loc[dimCountry['countryCode'] == 'TUR', 'countryName'] = 'Turkey'

# should now have no rows
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryID,countryCode,countryName


# 2.2. Transforming the data

In [369]:
# Dataframe for emission fact table

# organize data
fctEmission = combined_df.iloc[:, :-4]
fctEmission = fctEmission.drop('end_time', axis=1)
fctEmission['start_time'] = fctEmission['start_time'].astype(str).str[:4]
fctEmission.insert(0, 'emissionID', range(1, len(fctEmission) + 1))
fctEmission = fctEmission.reset_index(drop=True)
fctEmission

# rename columns
fctEmission = fctEmission.rename(columns={
    'iso3_country': 'countryID',
    'start_time':'yearID',
    'original_inventory_sector':'subsectorID',
    'gas':'gasID',
    'emissions_quantity':'emissionQuantity'
    })

# create dictionaries to map the dimension IDs to the fact table
dict_country = {dimCountry['countryCode'][i]:dimCountry['countryID'][i] for i in range(len(dimCountry))}
dict_year = {dimYear['yearName'][i]:dimYear['yearID'][i] for i in range(len(dimYear))}
dict_subsector = {dimSubSector['subsectorName'][i]:dimSubSector['subsectorID'][i] for i in range(len(dimSubSector))}
dict_gas = {dimGas['gasName'][i]:dimGas['gasID'][i] for i in range(len(dimGas))}

# map based on dimension tables
fctEmission['countryID'] = fctEmission['countryID'].map(dict_country)
fctEmission['yearID'] = fctEmission['yearID'].map(dict_year)
fctEmission['subsectorID'] = fctEmission['subsectorID'].map(dict_subsector)
fctEmission['gasID'] = fctEmission['gasID'].map(dict_gas)

# rearrange columns to match our dimensional model
fctEmission = fctEmission[['emissionID', 'countryID', 'subsectorID', 'yearID', 'gasID' ,'emissionQuantity']]

In [370]:
fctEmission.head(20)

,emissionID,countryID,subsectorID,yearID,gasID,emissionQuantity
0,1,1,4,6,1,0.0
1,2,1,4,6,2,0.0
2,3,2,4,6,1,0.0
3,4,2,4,6,2,0.0
4,5,3,4,6,1,0.0
5,6,3,4,6,2,0.0
6,7,4,4,6,1,0.0
7,8,5,4,6,1,0.0
8,9,5,4,6,3,0.0
9,10,5,4,6,2,0.0


# 3. Loading the data to the data warehouse

In [371]:
# export data as CSV files
combined_df.to_csv('transformed_data/combined_dataframe.csv', index=False)
dimSector.to_csv('transformed_data/dimSector.csv', index=False)
dimCountry.to_csv('transformed_data/dimCountry.csv', index=False)
dimGas.to_csv('transformed_data/dimGas.csv', index=False)
dimSubSector.to_csv('transformed_data/dimSubSector.csv', index=False)
dimYear.to_csv('transformed_data/dimYear.csv', index=False)
fctEmission.to_csv('transformed_data/fctEmission.csv', index=False)